In [ ]:
import os
from os import listdir
from os.path import join
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import random
from functions import _train, _forecast, _forecast_from_saved_model
from src.plots.systems import plot_forecast
from src.plots.systems import plot_system

from src.model import *

# FIX: Standardize data
# BUG: transient 0
# TODO: use fit with 1 epoch and then reset model states
# NOTE: Separate system generation to other repo
# TODO: save dt in params.json


folder = '/home/dionisio35/Documents/thesis/data/MG/16.8/'
data_file = join(folder, random.choice(listdir(folder)))
dt=0.5
steps = 10

transient = 1000
train_length = 20000
forecast_length = 2000

input_scaling = 0.5 # input sigma
reservoir_degree = 8 # numero de conexiones de nodos del reservoir
spectral_radius = 1.05
reservoir_sigma = 0.2 # sigma de los pesos del reservoir
rewiring = 1 # probabilidad de reconexion en reservoir Watts-Strogatz

# FIX
units = 1000 # Numero de nodos del reservoir
seed = 23970846
reservoir_activation = 'tanh'

regularization = 1e-08
leak_rate = 0.2 # Cuanta informacion se filtra hacia el siguiente estado 0-1(toda-nada)

input_initializer = 'InputMatrix'
input_bias_initializer = 'RandomUniform'
reservoir_initializer = 'WattsStrogatzNX'

In [ ]:
model = _train(
    data_file=data_file,
    model='ESN',

    units=units,
    steps=steps,
    transient=transient,
    train_length=train_length,

    input_initializer=input_initializer,
    input_bias_initializer=input_bias_initializer,
    reservoir_initializer=reservoir_initializer,
    reservoir_activation=reservoir_activation,

    input_scaling=input_scaling,
    leak_rate=leak_rate,
    spectral_radius=spectral_radius,
    rewiring=rewiring,
    reservoir_degree=reservoir_degree,
    reservoir_sigma=reservoir_sigma,
    regularization=regularization,
    
    seed=seed,
)

In [ ]:
for i in listdir(folder):
    file = join(folder, i)
    forecast, val_target = _forecast(
        trained_model=model,
        transient=transient,
        train_length=train_length,
        data_file=file,
        forecast_method='classic',
        forecast_length=forecast_length,
        steps=steps,
    )
    plot_forecast(forecast, val_target, show=True, dt=dt)